# 1. Initializations

## 1.1 General imports

In [ ]:
### Data management
import pandas as pd
import numpy as np
import random

### Machine Learning
# transformation
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
# models
import statsmodels.formula.api as smf
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
# resampling 
from imblearn.over_sampling import SMOTE, RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
# metrics and evaluation
from sklearn.model_selection import StratifiedKFold, cross_val_score, cross_validate
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from scipy.stats import chi2_contingency, probplot

### Data Viz
# graphical basics
import matplotlib.pyplot as plt
%matplotlib inline
# graphical seaborn
import seaborn as sns
# # graphical plotly
# import plotly.graph_objects as go
# import plotly.express as px
# # for jupyter notebook display management
# import plotly.io as pio
# pio.renderers.default = "notebook"


## 1.2 General dataframe functions

In [ ]:
import smartcheck.dataframe_common as dfc

# 2. Loading and Data Quality

## 2.1 Loading of data sets and general exploration

### 2.1.1 VELIB DISPO

In [ ]:
df_disp_velib_raw = dfc.load_dataset_from_config('velib_dispo_data', sep=';')

if df_disp_velib_raw is not None and isinstance(df_disp_velib_raw, pd.DataFrame):
    display(df_disp_velib_raw.head())
    dfc.log_general_info(df_disp_velib_raw)
    nb_first, nb_total = dfc.detect_and_log_duplicates_and_missing(df_disp_velib_raw)
    if nb_first != nb_total:
        print(dfc.duplicates_index_map(df_disp_velib_raw))
    df_disp_velib = dfc.normalize_column_names(df_disp_velib_raw)

#### Global Assessment

In [ ]:
df_disp_velib.info()
display(df_disp_velib.head())
df_cpt_velib_desc = df_disp_velib.select_dtypes(include=np.number).describe()
display(df_cpt_velib_desc)
df_cpt_velib_cr = df_disp_velib.select_dtypes(include=np.number).corr()
display(df_cpt_velib_cr)
dfc.display_variable_info(df_disp_velib, 5)

In [ ]:
# Analyse de la distribution d'une variable spécique en relation avec celle de son dataframe
ref_col = 'station_en_fonctionnement' # ici notre variable cible
dfc.analyze_by_reference_variable(df_disp_velib, ref_col)

In [ ]:
# Analyse croisée de la distribution d'une variable spécifique en fonction d'autres variables (quantitatives ou qualitatives) du dataframe
ref_col = 'station_en_fonctionnement' # ici notre variable cible
cross_columns = [ref_col] + ['borne_de_paiement_disponible', 'retour_velib_possible']
dfc.log_cross_distributions(
    df_disp_velib[cross_columns], 
    ref_col
)

In [ ]:
# Analyse croisée d'une variable en fonction d'une autre
ref_col = 'borne_de_paiement_disponible'
target_col = 'station_en_fonctionnement'
display(pd.crosstab(df_disp_velib[ref_col], df_disp_velib[target_col]))

In [ ]:
# Analyse croisée des d'une variable en fonction d'une autre (en conservant les NaN)
ref_col = 'station_opening_hours'
target_col = 'station_en_fonctionnement'
ref_cross_tab = pd.crosstab(df_disp_velib[ref_col], df_disp_velib[target_col], dropna=False, normalize=True)
display(ref_cross_tab)

# catégorisation via normalisation
ref_col_val_norm = np.where(
    df_disp_velib[ref_col].isin(ref_cross_tab[ref_cross_tab['OUI'] >= 0.8].index.tolist()), 
    1, 
    0
)
df_disp_velib[ref_col] = ref_col_val_norm
ref_cross_tab_norm = pd.crosstab(df_disp_velib[ref_col], df_disp_velib[target_col], dropna=False, normalize=True)
display(ref_cross_tab_norm)


In [ ]:
ref_col = 'retour_velib_possible'
target_col = 'station_en_fonctionnement'

# Génération des colonnes dummies pour ref_col
ref_col_dummies = pd.get_dummies(df_disp_velib[ref_col], prefix=ref_col)
print("Colonnes dummies générées :", list(ref_col_dummies.columns))

# Pour chaque modalité (dummy), tester son lien avec la variable cible
for col in ref_col_dummies:
    # Test du Chi-Deux
    cross_tab = pd.crosstab(df_disp_velib[target_col], ref_col_dummies[col])
    if cross_tab.shape[1] != 2:
        print(f"⚠️ Modalité [{col}] ignorée (1 seule valeur présente)")
        continue
    stat, p, _, _ = chi2_contingency(cross_tab)
    # V de Cramer
    V_Cramer = np.sqrt(
        stat/cross_tab.values.sum())
    # On affiche uniquement les variables significatives et dont le V de Cramer est supérieur à 0.1
    # Faible : Valeur autour de 0.1 ;
    # Moyenne : Valeur autour de 0.3 ;
    # Elevée : Valeur autour et supérieure à 0.5.
    # Lorsque la valeur du V de Cramer est très élevée (aux alentours de 0.8 et plus), on soupçonne généralement de la multicolinéarité.
    result = 'significative' if (p < 0.05) and (V_Cramer > 0.1) else 'NON signficative'
    print(f"Variable [{col}] {result} Vs [{target_col}]: p-value[{p:.5f}], V_Cramer[{V_Cramer:.5f}]")

In [ ]:
# Vérificationn de la répartition en loi normale de chaque données numérique
for col in df_disp_velib.select_dtypes(include='number').columns:
    probplot(df_disp_velib[col], dist="norm", plot=plt)
    plt.suptitle(f"Column {col}")
    plt.show()

### 2.1.2 VELIB COMPTAGE

In [ ]:
df_cpt_velib_raw = dfc.load_dataset_from_config('velib_comptage_data', sep=';')

if df_cpt_velib_raw is not None and isinstance(df_cpt_velib_raw, pd.DataFrame):
    display(df_cpt_velib_raw.head())
    dfc.log_general_info(df_cpt_velib_raw)
    nb_first, nb_total = dfc.detect_and_log_duplicates_and_missing(df_cpt_velib_raw)
    if nb_first != nb_total:
        print(dfc.duplicates_index_map(df_cpt_velib_raw))
    df_cpt_velib = dfc.normalize_column_names(df_cpt_velib_raw)

In [ ]:
df_cpt_velib.info()
display(df_cpt_velib.head())
df_cpt_velib_desc = df_cpt_velib.select_dtypes(include=np.number).describe()
display(df_cpt_velib_desc)
df_cpt_velib_cr = df_cpt_velib.select_dtypes(include=np.number).corr()
display(df_cpt_velib_cr)

## 2.2 Data quality refinement

### 2.2.2 VELIB COMPTAGE

In [ ]:
# Original backup and duplicates management
df_disp_velib_orig = df_disp_velib.copy()
df_disp_velib = df_disp_velib.drop_duplicates()

In [ ]:
# Exemple de modification localisée en fonction de la proximité à la médiane d'autre variables
# mask = (
#     (train['Gender'].isna()) &
#     (abs(train['Age'] - 30) > abs(train['Age'] - 41)) & # L’âge est plus proche de 41 que de 30
#     (train['Previously_Insured'] == 0) & # La personne n’était pas assurée auparavant
#     (train['Vehicle_Damage'] == 1) # Elle a subi un dommage sur son véhicule
# )
# train.loc[mask, 'Gender'] = 0

In [ ]:
# Exemple de modification par répartition spécifique entre deux valeurs 0 et 1
# proportion_tab = [0] * 55 + [1] * 45
# mask = (
#     (train['Gender'].isna()) &
# )
# train.loc[mask, 'Gender'] = train.loc[mask, 'Gender'].apply(lambda x: random.choice(proportion))

In [ ]:
# Preprocessing par scaling des données 
# - sans outlier ni distribution loi normale : min/max
# - sans outlier et distribution loi normale : standard
# - avec outlier : Robust 
mm_scaler = MinMaxScaler()
r_scaler = RobustScaler()
s_scaler = StandardScaler()

### 2.2.2 VELIB COMPTAGE

In [ ]:
# Original backup and duplicates management
df_cpt_velib_orig = df_cpt_velib.copy()
df_cpt_velib = df_cpt_velib.drop_duplicates()

## 2.3 Data combination and rework

# 2. Data Viz' and Analysis

## 2.1 General Data Viz'

## 2.2 Quantitative mono variable distribution

## 2.3 Qualitative mono variable distribution

## 2.4 Qualitative multi variable distribution

## 2.5 Quantitative multi variable correlation